In [1]:
import pandas as pd
import json

In [2]:
us_videos_df = pd.read_csv('./data/USvideos.csv')

Sorting the videos by video_id and date for later group by operations.

In [3]:
us_videos_df.sort_values(by=['video_id', 'trending_date'], ascending=True, inplace=True)

In [4]:
us_videos_df.head()

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
90,-2RVw2_QyxQ,17.14.11,2017 Champions Showdown: Day 3,Saint Louis Chess Club,27,2017-11-12T02:39:01.000Z,"Chess|""Saint Louis""|""Club""",67429,438,23,23,https://i.ytimg.com/vi/-2RVw2_QyxQ/default.jpg,False,False,False,The Saint Louis Chess Club hosts a series of f...
346,-2RVw2_QyxQ,17.15.11,2017 Champions Showdown: Day 3,Saint Louis Chess Club,27,2017-11-12T02:39:01.000Z,"Chess|""Saint Louis""|""Club""",69786,453,26,19,https://i.ytimg.com/vi/-2RVw2_QyxQ/default.jpg,False,False,False,The Saint Louis Chess Club hosts a series of f...
584,-2RVw2_QyxQ,17.16.11,2017 Champions Showdown: Day 3,Saint Louis Chess Club,27,2017-11-12T02:39:01.000Z,"Chess|""Saint Louis""|""Club""",71089,460,27,20,https://i.ytimg.com/vi/-2RVw2_QyxQ/default.jpg,False,False,False,The Saint Louis Chess Club hosts a series of f...
1404,-El8z4uy5Ac,17.21.11,The Best Movie Hell to End Up In - After Hours,Cracked,23,2017-11-20T17:00:00.000Z,"movie hells|""hercules""|""bedazzled""|""all dogs g...",327936,11643,584,1305,https://i.ytimg.com/vi/-El8z4uy5Ac/default.jpg,False,False,False,The gang gets together for a frank discussion ...
1621,-El8z4uy5Ac,17.22.11,The Best Movie Hell to End Up In - After Hours,Cracked,23,2017-11-20T17:00:00.000Z,"movie hells|""hercules""|""bedazzled""|""all dogs g...",522256,14312,1062,1553,https://i.ytimg.com/vi/-El8z4uy5Ac/default.jpg,False,False,False,The gang gets together for a frank discussion ...


Retrieving the last video of every group. The last video would be the latest video (as per trending date).

In [5]:
us_grouped_videos = us_videos_df.groupby(['video_id']).last().reset_index()

In [6]:
us_grouped_videos.head()

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
0,-2RVw2_QyxQ,17.16.11,2017 Champions Showdown: Day 3,Saint Louis Chess Club,27,2017-11-12T02:39:01.000Z,"Chess|""Saint Louis""|""Club""",71089,460,27,20,https://i.ytimg.com/vi/-2RVw2_QyxQ/default.jpg,False,False,False,The Saint Louis Chess Club hosts a series of f...
1,-El8z4uy5Ac,17.26.11,The Best Movie Hell to End Up In - After Hours,Cracked,23,2017-11-20T17:00:00.000Z,"movie hells|""hercules""|""bedazzled""|""all dogs g...",639982,16519,1243,1739,https://i.ytimg.com/vi/-El8z4uy5Ac/default.jpg,False,False,False,The gang gets together for a frank discussion ...
2,-JVITToppE0,17.17.11,Coach Taggart Monday Presser Ahead of Arizona,GoDucksdotcom,17,2017-11-13T20:41:45.000Z,"Oregon|""Ducks""|""college athletics""|""college fo...",704,10,2,5,https://i.ytimg.com/vi/-JVITToppE0/default.jpg,False,False,False,Coach Taggart addresses the media ahead of the...
3,-OK9i1_YYYc,17.16.11,Animal Adventure Park Giraffe Cam,Animal Adventure Park,15,2017-11-12T00:18:43.000Z,"Giraffe|""Zoo""|""New York""|""giraffe cam""|""Animal...",46260,2286,35,17,https://i.ytimg.com/vi/-OK9i1_YYYc/default.jpg,False,False,False,November 2017 Giraffe Cam Hours\n\nEnjoy your ...
4,-SIp6xSP7ds,17.28.11,Gayle King and Norah O'Donnell respond to Char...,CBS This Morning,25,2017-11-21T12:35:39.000Z,"video|""cbs""|""news""|""CBS This Morning""|""Charlie...",534125,2507,1201,1772,https://i.ytimg.com/vi/-SIp6xSP7ds/default.jpg,False,False,False,CBS This Morning co-hosts Norah O'Donnell and ...


Reading categories from json file

In [7]:
with open('./data/US_category_id.json') as data_file:
    data = json.load(data_file)

In [17]:
categories = []
for item in data['items']:
    category = {}
    category['category_id'] = int(item['id'])
    category['title'] = item['snippet']['title']
    categories.append(category)

In [18]:
categories_df = pd.DataFrame(categories)
categories_df.head()

,category_id,title
0,1,Film & Animation
1,2,Autos & Vehicles
2,10,Music
3,15,Pets & Animals
4,17,Sports


In [25]:
us_final_df = us_grouped_videos.merge(categories_df, on = ['category_id'])
us_final_df.rename(columns={'title_y': 'category', 'title_x': 'video_name'})

,video_id,trending_date,video_name,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,category
0,-2RVw2_QyxQ,17.16.11,2017 Champions Showdown: Day 3,Saint Louis Chess Club,27,2017-11-12T02:39:01.000Z,"Chess|""Saint Louis""|""Club""",71089,460,27,20,https://i.ytimg.com/vi/-2RVw2_QyxQ/default.jpg,False,False,False,The Saint Louis Chess Club hosts a series of f...,Education
1,-oXybog2IuI,17.21.11,24 Facts about Koalas - mental_floss List Show...,Mental Floss,27,2017-11-15T16:00:00.000Z,"john green|""mental floss""|""koalas""|""marsupial""...",38775,1373,16,140,https://i.ytimg.com/vi/-oXybog2IuI/default.jpg,False,False,False,A weekly show where knowledge junkies get thei...,Education
2,16W7c0mb-rE,17.24.11,Emergence – How Stupid Things Become Smart Tog...,Kurzgesagt – In a Nutshell,27,2017-11-16T15:01:58.000Z,"emergence|""ants""|""intelligence""|""ant""|""sum of ...",2032821,124607,1183,8577,https://i.ytimg.com/vi/16W7c0mb-rE/default.jpg,False,False,False,How can many stupid things combine to form sma...,Education
3,5WUDfviiKRE,17.28.11,二贵摔跤 - tienghoa.net,Tina Nguyen,27,2011-03-01T04:14:08.000Z,hanyuqiao,21342,107,312,201,https://i.ytimg.com/vi/5WUDfviiKRE/default.jpg,False,False,False,辽阔的乌珠穆沁草原是摔跤手的摇篮。这里摔跤的传统源远流长，盛名至今不衰。康熙五年(1666年...,Education
4,8-u5nd2GqNE,17.24.11,The Secret Protocol for When the Queen Dies,Half as Interesting,27,2017-11-16T15:30:00.000Z,"the|""secret""|""protocal""|""procedure""|""process""|...",1145464,28690,887,5083,https://i.ytimg.com/vi/8-u5nd2GqNE/default.jpg,False,False,False,Raise money for charity just by browsing the i...,Education
5,88kkrRv1UgI,17.25.11,What Happens to You if You Upload Your Mind to...,RealLifeLore,27,2017-11-17T16:37:31.000Z,"real life lore|""real life lore maps""|""real lif...",377704,14359,830,2056,https://i.ytimg.com/vi/88kkrRv1UgI/default.jpg,False,False,False,Check out Assassin's Creed Origins: http://bit...,Education
6,B4gXsobd_ao,17.14.11,What Are Diminutives – and Why We Like Them,The School of Life,27,2017-11-09T14:00:01.000Z,"the school of life|""school""|""life""|""education""...",79306,3809,127,670,https://i.ytimg.com/vi/B4gXsobd_ao/default.jpg,False,False,False,A diminutive is something you stick on the end...,Education
7,B5HORANmzHw,17.15.11,Founding An Inbreeding-Free Space Colony,SciShow,27,2017-11-12T22:00:01.000Z,"SciShow|""science""|""Hank""|""Green""|""education""|""...",286577,9606,248,1196,https://i.ytimg.com/vi/B5HORANmzHw/default.jpg,False,False,False,Thanks to 23AndMe for supporting SciShow. Thes...,Education
8,CxC161GvMPc,17.28.11,What is the tragedy of the commons? - Nicholas...,TED-Ed,27,2017-11-21T16:03:42.000Z,"TED|""TED-Ed""|""TED Education""|""TED Ed""|""Nichola...",468081,19382,390,1309,https://i.ytimg.com/vi/CxC161GvMPc/default.jpg,False,False,False,Check out our Patreon page: https://www.patreo...,Education
9,EsBjHXCG4Us,17.25.11,How Likely Are You to Die During a Black Frida...,The Infographics Show,27,2017-11-17T17:00:03.000Z,"black friday|""black friday sales""|""How Likely ...",527531,10200,540,2197,https://i.ytimg.com/vi/EsBjHXCG4Us/default.jpg,False,False,False,🐻 Check out our new channel: Fuzzy & Nutz 🐿️ ►...,Education


In [27]:
us_final_df.description[0]

'The Saint Louis Chess Club hosts a series of four matches November 9-14. Today features eight rounds of G/10, plus the G/30 games of Carlsen-Ding.  For the first time in recent history, each match features no delay or increment.  Join GMs Yasser Seirawan, Alejandro Ramirez, Maurice Ashley, Cristian Chirila, and WGM Jen Shahade for the move-by-move.\\n\\nUSChessChamps.com\\n2017.11.11'